# 1. Downloading data, clean and preprocess



In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd
from zipfile import ZipFile
import zipfile
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to /home/bptran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/bptran/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bptran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/bptran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/bptran/anaconda3/envs/sota/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torchinfo
import torchinfo

## 1.1 Loading data 

In [72]:
# Fetch the data
path = "data_full4.zip"

df = pd.DataFrame()
with zipfile.ZipFile(path) as z:
  for name in z.namelist():
    if name.endswith(".csv"):
      print(f'Loading data from {name}...')
      x = pd.read_csv(z.open(name))
      print(f'Loading completed from {name}...')
      df = pd.concat([df, x[['genre','plot']]],axis=0,ignore_index=True)
  print("Dataframe (df) ready to be used!")

df

Loading data from content/gdrive/MyDrive/CS5242_Project_Data_2/action/action_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data_2/action/action_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data_2/adventure/adventure_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data_2/adventure/adventure_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data_2/comedy/comedy_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data_2/comedy/comedy_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data_2/drama/drama_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data_2/drama/drama_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data_2/horror/horror_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data_2/horror/horror_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data_2/roma

,genre,plot
0,action,twentytwo year old tori coro gets involved in ...
1,action,a criminal gang discovers a genghis khan treas...
2,action,in 1890 london private detective sherlock holm...
3,action,jenny is a student studying abroad in london l...
4,action,neil is a former triad boss who has just been ...
...,...,...
3752,crime,journalist rex banner peter reynolds with the ...
3753,crime,smalltown girl jean lowell is about to wed far...
3754,crime,as recorded in a film magazine maggie the daug...
3755,crime,two women with bad taste in men are thrown tog...


In [73]:
df.groupby('genre').count()

,plot
genre,
action,484
adventure,467
comedy,462
crime,425
drama,434
fantasy,206
historical,246
horror,561
romance,156


In [88]:
#new_df = df[~df['genre'].isin(["action", "adventure", "drama", "fantasy", "historical", "romance"])]
new_df = df[~df['genre'].isin(["action", "adventure", "comedy", "drama", "historical", "horror"])]

In [89]:
new_df.groupby('genre').count()

,plot
genre,
crime,425
fantasy,206
romance,156
scifi,316


In [90]:
df[df['genre'] == 'comedy'].iloc[1]['plot']

'before undergoing an operation at a hospital in chile peter ingersoll explains the origin of his unusual condition peter a california insurance salesman learns that he only has a short time left to live with his wifes encouragement peter embarks on an epic fishing excursion accruing 100000 of charges on his credit card however dr carter contacts peter to inform him that he was misdiagnosed and is not dying peter is urged by dr carter to fake his death to avoid paying the large debt and to permit his wife to collect a 150000 lifeinsurance policy after seven years when the statute of limitations expires peter can reappear peter discovers that the entire plan was a scheme concocted by his wife and dr carter who are having an affair he is determined to sabotage their plans but instead finds himself on the operating table in chile with a marlin piercing his chest'

## 1.2. Clean data

In [91]:
#Commence data cleaning, split data into X and Y
plot_list, genre_list = new_df['plot'] , new_df['genre']

#Load, pre-process and clean data before splitting into train and test set
lemma = WordNetLemmatizer()


## 1.3. Tokenization, embeddings

In [92]:
def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        padd_tokenized_sent = tokenized_sent + ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in padd_tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

def encode_no_pad(tokenized_texts, word2idx):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return input_ids

In [93]:
from tqdm import tqdm

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    if "glove" in fname:
        d = len(fin.readline().split()) - 1
    elif "crawl" in fname:
        n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [94]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(plot_list)

# Load pretrained vectors, please put 'glove.6B.300d.txt' in current folder to be run.
# 'glove.6B.300d.txt' is downloaded at https://www.kaggle.com/datasets/thanakomsn/glove6b300dtxt
embeddings = load_pretrained_vectors(word2idx, "glove.6B.300d.txt")
embeddings = torch.tensor(embeddings)

Tokenizing...

Loading pretrained vectors...


399999it [00:09, 43693.57it/s]


There are 22530 / 25763 pretrained vectors found.


## 1.4. Preparing dataset and dataloader

In [95]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, Dataset,
                              SequentialSampler)

from typing import List

class FilmGenres(Dataset):
    def __init__(self, inputs: List, labels: np.ndarray):
        '''
        train: True if dataset for training, otherwise False is for testing
        zip_file_path is path containing the plot films
        '''
        self.list_of_plot = inputs
        self.array_of_labels = labels


    def __len__(self):
        return len(self.list_of_plot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if idx >= len(self):
            raise IndexError

        sample = torch.tensor(self.list_of_plot[idx]), torch.tensor(self.array_of_labels[idx])

        return sample

def data_loader(train_inputs, val_inputs, train_labels, val_labels,batch_size=1):
    """Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """


    # Create DataLoader for training data
    train_data = FilmGenres(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = FilmGenres(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [96]:
from sklearn.model_selection import train_test_split

# Train Test Split

# 1. Noting here, if use batch_size > 1, then must use encode
# 2. If use batch_size = 1, then can use encode or encode_no_pad

# train_inputs is list of list of integer number where number is encoded of token, and train_labels is numpy array
# encode tokens

input_ids = encode_no_pad(tokenized_texts, word2idx) # For batch_size = 1
#input_ids = encode(tokenized_texts, word2idx, max_len) # For batch_size > 1

#Transformation for Y#
#create dictionary to retain information on different classes
#action, adventure, comedy, crime, drama, fantasy, historical, horror, romance, scifi
#y_info = {'action': 0, 'adventure':1,'comedy':2,'crime':3, 'drama':4,'fantasy':5,'historical':6,'horror':7,'romance':8,'scifi':9}
y_info = {'crime':0, 'fantasy':1, 'scifi':2, 'romance':3}

Y = genre_list.apply(lambda y: y_info[y.lower()])

#Transformation for Y, one hot encode for multiclass
Y = Y.to_numpy() #In order to use reshape below

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, Y, test_size=0.1, random_state=42)

In [97]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=1)

# 2. Common train / test procedure for training

In [98]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3060 Ti


In [99]:
import random
import time
from tqdm import tqdm

# Specify loss function
loss_fn = nn.CrossEntropyLoss() 


def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        running_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(tqdm(train_dataloader)):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)
            # Perform a forward pass. This will return logits.
            model_outputs = model(b_input_ids) #added by myself

            # compute loss
            loss = loss_fn(model_outputs,b_labels) #added by myself

            # Zero out any previously calculated gradients
            optimizer.zero_grad() #added by myself

            # Perform a backward pass to calculate gradients
            loss.backward() #added by myself

            # Update parameters
            optimizer.step()

            # Compute loss and accumulate the loss values
            running_loss += loss.item()
            

        # Calculate the average loss over the entire training data
        avg_train_loss = running_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, f'{PATH}/cnn.pt')

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            
            with open(f'{PATH}/log.txt', 'a') as f:
                f.write(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}\n")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

# 2. CNN

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(x_fc)

        return logits

In [101]:
import torch.optim as optim

def initilize_cnn_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=num_classes,
                        dropout=dropout)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    # For cnn4
#     optimizer = optim.Adam(cnn_model.parameters(),
#                                lr=learning_rate)

    return cnn_model, optimizer

In [102]:
# 2. CNN model: golve.6b.100d pretrained word vectors are fine-tuned during training.
# Accurcy for this model is 57.99%
PATH = "cnn_newdata"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 4, 5]
num_filters = [128, 256, 512]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.01)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:01<00:00, 600.78it/s]


   1    |   1.262801   |  1.191410  |   45.05   |   3.42   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:01<00:00, 628.80it/s]


   2    |   1.039531   |  1.052465  |   57.66   |   2.24   


...

  98    |   0.002543   |  0.532004  |   81.08   |   1.63   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:01<00:00, 629.30it/s]


  99    |   0.002506   |  0.531986  |   81.08   |   1.65   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:01<00:00, 628.82it/s]


  100   |   0.002463   |  0.533609  |   81.08   |   1.65   


Training complete! Best accuracy: 84.68%.


In [103]:
from torchinfo import summary
summary(model, (1, 100), batch_dim=None, dtypes = [torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
CNN_NLP                                  [1, 4]                    --
├─Embedding: 1-1                         [1, 100, 300]             (7,728,900)
├─ModuleList: 1-2                        --                        --
│    └─Conv1d: 2-1                       [1, 128, 98]              115,328
│    └─Conv1d: 2-2                       [1, 256, 97]              307,456
│    └─Conv1d: 2-3                       [1, 512, 96]              768,512
├─Linear: 1-3                            [1, 4]                    3,588
Total params: 8,923,784
Trainable params: 1,194,884
Non-trainable params: 7,728,900
Total mult-adds (M): 122.64
Input size (MB): 0.00
Forward/backward pass size (MB): 1.17
Params size (MB): 66.61
Estimated Total Size (MB): 67.78

In [104]:
# just a bit analysis


# 2. RNN

In [105]:
class RNNClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(RNNClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.layer1 = self.embedding
        self.layer2 = nn.RNN(       embed_dim , embed_dim*2  , batch_first=True)
        self.layer3 = nn.RNN(       embed_dim*2 , embed_dim*2  , batch_first=True)

        self.layer4 = nn.Linear(    embed_dim*2 , embed_dim  )
        self.layer5 = nn.Linear(    embed_dim , num_classes)

        
    def forward(self, word_seq ):
        g_seq               =   self.layer1( word_seq )
        h_seq, h_final     =   self.layer2( g_seq  )
        h_seq, h_final     =   self.layer3( h_seq  )

        linear1 = F.relu(self.layer4(h_final))
        score_seq           =   self.layer5( linear1 )
        score_seq = score_seq.squeeze(1) # Unsequeeze the seq dimension
        return score_seq 

In [106]:
def initilize_rnn_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    rnn_model = RNNClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    rnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(rnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return rnn_model, optimizer

In [107]:
# Best 34.57%
set_seed(42)
PATH = "rnn1_newdata"
os.makedirs(PATH, exist_ok=True)
rnn_model, optimizer = initilize_rnn_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.05)
train(rnn_model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:15<00:00, 65.75it/s]


   1    |   1.284017   |  1.549770  |   43.24   |   15.87  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:12<00:00, 79.55it/s]


   2    |   1.013382   |  1.137122  |   54.95   |   13.32  


...

  98    |   0.001474   |  5.032997  |   50.45   |   12.86  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:12<00:00, 81.58it/s]


  99    |   0.001473   |  5.035322  |   50.45   |   12.65  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:12<00:00, 79.34it/s]


  100   |   0.001471   |  5.047675  |   50.45   |   13.02  


Training complete! Best accuracy: 54.95%.


In [108]:
from torchinfo import summary
summary(rnn_model, (1, 100), batch_dim=None, dtypes = [torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
RNNClassification                        [1, 4]                    --
├─Embedding: 1-1                         [1, 100, 300]             (7,728,900)
├─RNN: 1-2                               [1, 100, 600]             541,200
├─RNN: 1-3                               [1, 100, 600]             721,200
├─Linear: 1-4                            [1, 1, 300]               180,300
├─Linear: 1-5                            [1, 1, 4]                 1,204
Total params: 9,172,804
Trainable params: 1,443,904
Non-trainable params: 7,728,900
Total mult-adds (M): 134.15
Input size (MB): 0.00
Forward/backward pass size (MB): 1.20
Params size (MB): 36.69
Estimated Total Size (MB): 37.89

# 3. LSTM

In [109]:
class LSTMClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(LSTMClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.layer1 = self.embedding
        self.layer2 = nn.LSTM(       embed_dim , embed_dim *2 , batch_first=True)
        self.layer3 = nn.LSTM(       embed_dim *2, embed_dim *2 , batch_first=True)

        self.layer4 = nn.Linear(    embed_dim *2, embed_dim   )
        self.layer5 = nn.Linear(    embed_dim, num_classes)

        
    def forward(self, word_seq ):
        g_seq               =   self.layer1( word_seq )
        h_seq, (h_final,  c_final)     =   self.layer2( g_seq  )
        h_seq, (h_final,  c_final)     =   self.layer3( h_seq  )

        linear1 = F.relu(self.layer4(h_final))
        score_seq           =   self.layer5( linear1 )
        score_seq = score_seq.squeeze(1) # Unsequeeze the seq dimension
        return score_seq 
        

In [110]:
def initilize_lstm_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    model = LSTMClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [111]:
set_seed(42)
PATH = "lstm1_newdata"
os.makedirs(PATH, exist_ok=True)
model, optimizer = initilize_lstm_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.01)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:32<00:00, 30.42it/s]


   1    |   1.370660   |  1.349485  |   38.74   |   34.24  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:32<00:00, 30.49it/s]


   2    |   1.326314   |  1.328082  |   38.74   |   34.10  


...

  58    |   0.322031   |  1.539698  |   63.96   |   35.94  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:34<00:00, 28.84it/s]


  59    |   0.331717   |  1.539625  |   61.26   |   36.06  


 33%|███████████████████████████████████████████████████████████████▍                                                                                                                                 | 326/992 [00:11<00:22, 29.42it/s]


KeyboardInterrupt: 

# 4. RNN with attention-based model

In [112]:
class RNNAttentionClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(RNNAttentionClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.encoder = nn.RNN(       embed_dim , embed_dim  , batch_first=True)

        self.linear1 = nn.Linear(embed_dim, embed_dim)
        
        self.adaptivePool1d = nn.AdaptiveAvgPool1d(1)

        self.linear2 = nn.Linear(    embed_dim *2, embed_dim)
        self.linear3 = nn.Linear(    embed_dim, num_classes)

        
    def forward(self, word_seq ):
        embed_out               =   self.embedding( word_seq )
        h_seq, h_final     =   self.encoder( embed_out  )

        vprime = self.linear1(h_final) # Squeeze the len axis to have V'
        alpha = torch.bmm(h_seq, torch.swapaxes(vprime, 1, 2))
        score = torch.nn.functional.softmax(alpha, dim=1) 
        c_w = torch.mul(h_seq, score)
        c_concat = torch.concat([c_w, h_seq], dim=-1) # batch_size, seq, dims
        
        #linear1 = F.relu(self.linear2(c_concat.mean(dim=1)))
        z = F.relu(self.adaptivePool1d(torch.swapaxes(c_concat, 1, 2)))

        score_seq           =   F.relu(self.linear2( z.squeeze(-1) ))
        score_seq = F.relu(self.linear3(score_seq))
        return score_seq 

In [113]:
model = RNNAttentionClassification(len(word2idx), 300, embeddings.float(), True, 10)
from torchinfo import summary
summary(model, [(1,3)], batch_dim=None, dtypes=[torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
RNNAttentionClassification               [1, 10]                   --
├─Embedding: 1-1                         [1, 3, 300]               (7,728,900)
├─RNN: 1-2                               [1, 3, 300]               180,600
├─Linear: 1-3                            [1, 1, 300]               90,300
├─AdaptiveAvgPool1d: 1-4                 [1, 600, 1]               --
├─Linear: 1-5                            [1, 300]                  180,300
├─Linear: 1-6                            [1, 10]                   3,010
Total params: 8,183,110
Trainable params: 454,210
Non-trainable params: 7,728,900
Total mult-adds (M): 8.54
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 32.73
Estimated Total Size (MB): 32.75

In [114]:
def initilize_rnnattention_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    model = RNNAttentionClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [115]:
# Best accuracy 49.07%.
PATH = "rnn_att_newdata"
os.makedirs(PATH, exist_ok=True)

set_seed(42)
rnn_model, optimizer = initilize_rnnattention_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.05)
train(rnn_model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 166.19it/s]


   1    |   1.303255   |  1.125047  |   38.74   |   6.36   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 166.56it/s]


   2    |   1.123248   |  1.051995  |   38.74   |   6.26   


...

  68    |   0.391197   |  1.221525  |   53.15   |   6.24   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 171.85it/s]


  69    |   0.389255   |  1.381794  |   58.56   |   6.08   


 29%|███████████████████████████████████████████████████████▋                                                                                                                                        | 288/992 [00:01<00:04, 172.79it/s]


KeyboardInterrupt: 

# 5. LSTM with Attention



In [116]:
class LSTMAttentionClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(LSTMAttentionClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.encoder = nn.LSTM(       embed_dim , embed_dim  , batch_first=True)

        self.linear1 = nn.Linear(embed_dim, embed_dim)
        
        self.adaptivePool1d = nn.AdaptiveAvgPool1d(1)

        self.linear2 = nn.Linear(    embed_dim *2, embed_dim)
        self.linear3 = nn.Linear(    embed_dim, num_classes)

        
    def forward(self, word_seq ):
        embed_out               =   self.embedding( word_seq )
        h_seq, (h_final, c_final)     =   self.encoder( embed_out  )

        vprime = self.linear1(h_final) # Squeeze the len axis to have V'
        alpha = torch.bmm(h_seq, torch.swapaxes(vprime, 1, 2))
        score = torch.nn.functional.softmax(alpha, dim=1) 
        c_w = torch.mul(h_seq, score)
        c_concat = torch.concat([c_w, h_seq], dim=-1) # batch_size, seq, dims
        
        #linear1 = F.relu(self.linear2(c_concat.mean(dim=1)))
        z = F.relu(self.adaptivePool1d(torch.swapaxes(c_concat, 1, 2)))

        score_seq           =   F.relu(self.linear2( z.squeeze(-1) ))
        score_seq = F.relu(self.linear3(score_seq))
        return score_seq 

In [117]:
def initilize_lstmattention_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    model = LSTMAttentionClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [124]:
# 49.07%.
PATH = 'lstmattention_newdata'
os.makedirs(PATH, exist_ok=True)
set_seed(42)
model, optimizer = initilize_lstmattention_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(well b+ i),
                                            learning_rate=0.05)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:10<00:00, 99.19it/s]


   1    |   1.382648   |  1.380603  |   26.13   |   10.71  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:10<00:00, 95.81it/s]


   2    |   1.325018   |  1.217837  |   45.95   |   11.21  


...

  98    |   0.080740   |  1.432916  |   76.58   |   10.48  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:09<00:00, 99.63it/s]


  99    |   0.093408   |  1.187702  |   77.48   |   10.46  


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:09<00:00, 99.53it/s]


  100   |   0.082567   |  1.303984  |   74.77   |   10.46  


Training complete! Best accuracy: 80.18%.


# 6. Transformer

In [118]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(num_heads=num_heads, embed_dim=embed_dim, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim ,embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(normalized_shape = [embed_dim], eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape = [embed_dim], eps=1e-6)
        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, inputs):
        attn_output, _ = self.att(inputs, inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [119]:
class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, pretrained_embedding, freeze_embedding):
        super(TokenAndPositionEmbedding, self).__init__()

        if pretrained_embedding is not None:
            self.token_emb = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.token_emb = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)


        self.pos_emb = nn.Embedding(num_embeddings=maxlen, embedding_dim=embed_dim)

    def forward(self, x):
        maxlen = x.size()[-1]
        positions = torch.arange(start=0, end=maxlen, step=1).to(device)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [120]:
class TransformerClassification(nn.Module):
  def __init__(self, maxlen, vocab_size, embed_dim, pretrained_embedding, freeze_embedding, num_heads, ff_dim, num_classes):
    super(TransformerClassification, self).__init__()

    self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim, pretrained_embedding, freeze_embedding)
    self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    self.averagePool = nn.AdaptiveAvgPool1d(output_size=1)
    self.sequential = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(embed_dim, 20),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(20, num_classes)
    )
    

  def forward(self, x):
    x = self.embedding_layer(x)
    #print(x.size())
    x = self.transformer_block(x)
    #print(x.size())

    # Swap from (N, L, features) to (N, features, L)
    x = torch.swapaxes(x, 1, 2)
    x = self.averagePool(x)
    x = x.squeeze(-1)
    #print(x.size())

    x = self.sequential(x)
    return x



In [121]:
#!pip install torchinfo
model = TransformerClassification(pretrained_embedding=embeddings.float(),
                        freeze_embedding=True,
                        vocab_size=len(word2idx),
                        embed_dim=300,
                        maxlen = max_len,
                        num_classes=len(np.unique(Y)),
                        num_heads = 2,
                        ff_dim = 300)
from torchinfo import summary
summary(model.to(device), [(1,100)], batch_dim=None, dtypes=[torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
TransformerClassification                [1, 4]                    --
├─TokenAndPositionEmbedding: 1-1         [1, 100, 300]             --
│    └─Embedding: 2-1                    [100, 300]                696,300
│    └─Embedding: 2-2                    [1, 100, 300]             (7,728,900)
├─TransformerBlock: 1-2                  [1, 100, 300]             --
│    └─MultiheadAttention: 2-3           [1, 100, 300]             361,200
│    └─Dropout: 2-4                      [1, 100, 300]             --
│    └─LayerNorm: 2-5                    [1, 100, 300]             600
│    └─Sequential: 2-6                   [1, 100, 300]             --
│    │    └─Linear: 3-1                  [1, 100, 300]             90,300
│    │    └─ReLU: 3-2                    [1, 100, 300]             --
│    │    └─Linear: 3-3                  [1, 100, 300]             90,300
│    └─Dropout: 2-7                      [1, 100, 300]   

In [122]:
def initilize_transformer_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim = 300,
                    num_classes=2,
                    maxlen = 10000,
                    learning_rate=0.01,
                    num_heads = 2,
                    ff_dim = 300
                    ):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate transformer model
    model = TransformerClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        maxlen = maxlen,
                        num_classes=num_classes,
                        num_heads = num_heads,
                        ff_dim = ff_dim)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [123]:
# 44.98% accuracy
set_seed(42)
PATH = "transformer_newdata"
os.makedirs(PATH, exist_ok=True)
model, optimizer = initilize_transformer_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=False,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            maxlen = max_len,
                                            learning_rate=0.1,
                                            num_heads = 10,
                                            ff_dim = 300)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 180.29it/s]


   1    |   1.281723   |  1.111678  |   53.15   |   5.74   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 188.68it/s]


   2    |   0.975640   |  0.870508  |   61.26   |   6.10   


...

  98    |   0.001822   |  3.815817  |   73.87   |   5.48   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 192.22it/s]


  99    |   0.002460   |  3.728880  |   73.87   |   5.24   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:05<00:00, 180.17it/s]


  100   |   0.006529   |  3.445806  |   75.68   |   5.57   


Training complete! Best accuracy: 78.38%.
